[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Cat-vs-Dog-Classifier-Using-CNN/blob/master/Cat_vs_Dog_Classifier_using_scratch_CNN.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmplougva10/pubring.gpg' created
gpg: /tmp/tmplougva10/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
os.chdir("drive/Cat vs Dog Classifier")
import tensorflow as tf
import numpy as np
import cv2

In [0]:
train_data = np.load('train_data.npy')
test_data = np.load('test_data.npy')


In [5]:
print(np.shape(train_data))

(25000, 2)


In [0]:
training_data = train_data[:23552]
validation_data = train_data[23552:25000]
training_x = []
training_y = []
validation_y = []
validation_x = []
for i in range(23552):
  training_x.append(training_data[i][0])
  training_y.append(training_data[i][1])
for i in range(1448):
  validation_x.append(validation_data[i][0])
  validation_y.append(validation_data[i][1])

In [0]:
os.chdir('/content')
!git clone https://github.com/mixuala/colab_utils
import colab_utils.tboard
ROOT = %pwd
print(ROOT)
LOG_DIR = os.path.join(ROOT, 'gaurav')
print(LOG_DIR)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

In [0]:
tf.reset_default_graph()
hm_epochs = 10
batch_size = 1024

mode = tf.placeholder(dtype = tf.bool, name = 'mode')

x = tf.placeholder(dtype = tf.float32, shape = [None, 50, 50, 3], name = 'input_labels')
y = tf.placeholder(dtype = tf.int32, shape = [None, 2], name = 'output_labels')
global_step = tf.Variable(0, trainable = False, name = 'global_step')
learning_rate = tf.placeholder('float')

In [0]:
def CNN_model(x, mode):
  with tf.name_scope('CNN_network'):
    input_layer = tf.reshape(x, [-1, 50, 50, 3])
    
    conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=3,strides = 1,
                             padding="same", activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding = "same")
    
    conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=3, strides = 1,
                             padding="same", activation=tf.nn.relu)                             
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=3, strides=2, padding = "same")
    
    conv3 = tf.layers.conv2d(inputs=pool2, filters=256, kernel_size=3,
                             padding="same", activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=2, strides=2, padding = "valid")
    
    pool3_flat = tf.layers.flatten(pool3)
    dense1 = tf.layers.dense(inputs=pool3_flat, units=256, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense1, rate= .2, training=mode)

    logits = tf.layers.dense(inputs=dropout, units=2)

    return logits

In [0]:

prediction = CNN_model(x, mode)

with tf.name_scope('cost'):
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
summary = tf.summary.scalar('total_cost', cost)

with tf.name_scope('optimizer'):
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost, global_step = global_step)

merged = tf.summary.merge_all()

sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
train_writer = tf.summary.FileWriter('./',sess.graph)
k = 0


In [0]:
for i in range(hm_epochs):
  epoch_loss = 0
  starting  = 0
  end = batch_size
  while end < 23000:
    epoch_x = training_x[starting : end]
    epoch_y = training_y[starting : end]

    o, c, m = sess.run([optimizer, cost, merged], feed_dict = {x : epoch_x, y : epoch_y, mode : True, learning_rate : .001})
    epoch_loss += c
    train_writer.add_summary(m, k)
    k += 1
    starting = end
    end += batch_size
  print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)
saver.save(sess, './model')  




In [0]:
correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
print('Accuracy:',sess.run(accuracy, feed_dict = {x:validation_x, y:validation_y, mode : False})) 